In [0]:
%run ./IMPORTS

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
#create table s_info_corridas_dia
spark.sql("""
    CREATE TABLE IF NOT EXISTS s_info_corridas_dia (
        dt_refe STRING,
        qt_corr INT,
        qt_corr_neg INT,
        qt_corr_pess INT,
        vl_max_dist DOUBLE,
        vl_min_dist DOUBLE,
        vl_avg_dist DOUBLE,
        qt_corr_reuni INT,
        qt_corr_nao_reuni INT,
        dt_evento TIMESTAMP
    )
""")
print("Tabela criada com sucesso b_info_transporte")

Tabela criada com sucesso b_info_transporte


In [0]:
df_trans = spark.read.table("b_info_transporte")
df_trans.limit(10).display()

data_inicio,data_fim,categoria,local_inicio,local_fim,distancia,proposito
07-21-2016 17:42,07-21-2016 17:51,Negocio,Edgehill Farms,Burtrose,23,null
07-21-2016 18:27,07-21-2016 18:42,Negocio,Cary,Morrisville,37,null
07-21-2016 19:30,07-21-2016 19:39,Negocio,Morrisville,Cary,29,null
07-22-2016 10:42,07-22-2016 10:53,Negocio,Cary,Morrisville,38,null
07-22-2016 11:11,07-22-2016 11:25,Negocio,Morrisville,Cary,51,null
07-22-2016 11:37,07-22-2016 12:00,Negocio,Hazelwood,Lexington Park at Amberly,91,null
07-22-2016 12:14,07-22-2016 12:31,Negocio,Cary,Durham,8,null
07-22-2016 13:21,07-22-2016 13:42,Negocio,Durham,Cary,99,Reunião
07-22-2016 14:27,07-22-2016 14:43,Negocio,Cary,Morrisville,61,Alimentação
07-22-2016 15:49,07-22-2016 16:22,Negocio,Morrisville,Cary,122,null


In [0]:
condiction = to_date(substring(col("data_inicio"),1,10),"MM-dd-yyyy")

df_raw = df_trans.withColumns({"dt_refe":condiction,
                               "qt_corr":count("*").over(Window.partitionBy(condiction)),
                               "qt_corr_neg":count(when(col("proposito").isNull(),1)).over(Window.partitionBy(condiction)),
                               "qt_corr_pess":count(col("proposito").isin(["Caridade","Parada temporária","Alimentação"])).over(Window.partitionBy(condiction)),
                               "vl_max_dist":max(col("distancia")).over(Window.partitionBy(condiction)),
                               "vl_min_dist":min(col("distancia")).over(Window.partitionBy(condiction)),
                               "vl_avg_dist":round(avg(col("distancia")).over(Window.partitionBy(condiction)),2),
                               "qt_corr_reuni":count(when(col("proposito") == "Reunião",1)).over(Window.partitionBy(condiction)),
                               "qt_corr_nao_reuni":count(when(col("proposito") != "Reunião",1)).over(Window.partitionBy(condiction)),
                               
    }).select(col("dt_refe"),
              col("qt_corr"),
              col("qt_corr_neg"),
              col("qt_corr_pess"),
              col("vl_max_dist"),
              col("vl_min_dist"),
              col("vl_avg_dist"),
              col("qt_corr_reuni"),
              col("qt_corr_nao_reuni"))
    
df_final = df_raw.write.mode("overwrite").option("overwriteSchema",True).saveAsTable("s_info_corridas_dia")

In [0]:
df_raw = df_trans.withColumn("dt_refe",to_date(substring(col("data_inicio"),1,10),"MM-dd-yyyy"))

df_agreg = df_raw.groupBy("dt_refe").agg(count("*").alias("qt_corr"),count(when(col("proposito").isNull(),1)).alias("qt_corr_neg"),
                                         count(col("proposito").isin(["Caridade","Parada temporária","Alimentação"])).alias("qt_corr_pess"),
                                         max(col("distancia")).alias("vl_max_dist"),
                                         min(col("distancia")).alias("vl_min_dist"),
                                         round(avg(col("distancia")),2).alias("vl_avg_dist"),
                                         count(when(col("proposito") == "Reunião",1)).alias("qt_corr_reuni"),
                                         count(when(col("proposito") != "Reunião",1)).alias("qt_corr_nao_reuni"))
                             
# uma linha para cada data
df_agreg.write.mode("overwrite").option("overwriteSchema",True).saveAsTable("s_info_corridas_dia")

In [0]:
%sql
select * from s_info_corridas_dia

dt_refe,qt_corr,qt_corr_neg,qt_corr_pess,vl_max_dist,vl_min_dist,vl_avg_dist,qt_corr_reuni,qt_corr_nao_reuni
2016-02-05,2,0,2,104,104,104.0,2,0
2016-04-07,2,0,2,61,61,61.0,0,2
2016-06-29,6,4,2,104,16,64.0,2,0
2016-07-06,5,1,4,99,31,65.0,1,3
2016-04-15,3,0,3,152,14,95.0,1,2
2016-02-17,4,0,4,214,5,95.75,0,4
2016-01-01,1,0,1,51,51,51.0,0,1
2016-07-14,3,2,1,1953,33,701.33,0,1
2016-04-22,4,0,4,225,10,118.75,4,0
2016-03-31,2,0,2,164,161,162.5,0,2
